In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
analytic_dataset = spark.sql('select * from workarea.us_export_propensity_analytic_dataset')

smad = spark.sql('select duns_nbr as duns, alph_terr_code as state, prim_sic_code as sic4, load_year as append_year, load_month as append_month from workarea.smad where load_year in (2016, 2017) and load_month = 9')

smad = smad.withColumn('sic2', (smad.sic4.cast(IntegerType()) / 100))
smad = smad.withColumn('sic2d', lit(11))
smad = smad.withColumn('sic2d', when((col('sic2') > 0) & (col('sic2') <= 9), 1).otherwise(col('sic2d'))) # Agriculture, Forestry, Fishing
smad = smad.withColumn('sic2d', when((col('sic2') > 9) & (col('sic2') <= 14), 2).otherwise(col('sic2d'))) # Mining
smad = smad.withColumn('sic2d', when((col('sic2') > 14) & (col('sic2') <= 17), 3).otherwise(col('sic2d'))) # Construction
smad = smad.withColumn('sic2d', when((col('sic2') > 19) & (col('sic2') <= 39), 4).otherwise(col('sic2d'))) # Manufacturing
smad = smad.withColumn('sic2d', when((col('sic2') > 39) & (col('sic2') <= 49), 5).otherwise(col('sic2d'))) # Transportation & Public Utilities
smad = smad.withColumn('sic2d', when((col('sic2') > 49) & (col('sic2') <= 51), 6).otherwise(col('sic2d'))) # Wholesale Trade
smad = smad.withColumn('sic2d', when((col('sic2') > 51) & (col('sic2') <= 59), 7).otherwise(col('sic2d'))) # Retail Trade
smad = smad.withColumn('sic2d', when((col('sic2') > 59) & (col('sic2') <= 67), 8).otherwise(col('sic2d'))) # Finance, Insurance, Real Estate
smad = smad.withColumn('sic2d', when((col('sic2') > 69) & (col('sic2') <= 89), 9).otherwise(col('sic2d'))) # Services
smad = smad.withColumn('sic2d', when((col('sic2') > 90) & (col('sic2') <= 97), 0).otherwise(col('sic2d'))) # Public Administration

analytic_dataset = analytic_dataset.join(smad, on = ['duns', 'append_year', 'append_month'], how = 'left')

analytic_dataset = analytic_dataset.withColumn('sic2d', when(col('sic2d').isNull(), 11).otherwise(col('sic2d')))

In [ ]:
display(analytic_dataset.groupBy('append_year').agg(count('duns'), mean('export')))

In [ ]:
display(analytic_dataset.groupBy('sic2d').agg(count('duns'), mean('export')))

In [ ]:
display(analytic_dataset.groupBy('state').agg(count('duns'), mean('export')))